In [1]:
import pandas as pd
import numpy as np
import os
import math
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.ensemble import RandomForestClassifier
import scanpy as sc
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV

In [2]:
#Import a dataset
dir_path = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Influenza/Final analysis_headspace/2023_08_19_Breath_PR8_multiplex" 
dir_list = sorted(os.listdir(dir_path))

In [3]:
#import excel file using panda as df
name = dir_path + '/zscore_10min_code.xlsx'
df = pd.read_excel(name)

In [4]:
#Convert sample type to numpy and assign sample as 0 and 1
type = df.iloc[:,0]
type = type.to_numpy()

assign_type = []
for i in type:
    if 'H' in i:
        assign_type.append(0)
    elif 'R' in i:
        assign_type.append(1)

In [5]:
#Classify either with one sensor or multiplex
ns1_1 = df.iloc[:,1]
ns1_2 = df.iloc[:,2]
ns1_3 = df.iloc[:,3]
ns1_4 = df.iloc[:,4]
ns1_5 = df.iloc[:,5]

ns1_all = df.iloc[:,1:6]

ns1_1 = ns1_1.to_numpy().reshape(-1,1)
ns1_2 = ns1_2.to_numpy().reshape(-1,1)
ns1_3 = ns1_3.to_numpy().reshape(-1,1)
ns1_4 = ns1_4.to_numpy().reshape(-1,1)
ns1_5 = ns1_5.to_numpy().reshape(-1,1)

ns1_all = ns1_all.to_numpy()

In [6]:
#import the second dataset
dir_path2 = "C://Users/chris/OneDrive/Desktop/MIT postdoc project/ABN project/Influenza/Final analysis_headspace/2023_0913_0916__Breath_PR8_day3_day6_multiplex_longitudinal/Day6" 
dir_list2 = sorted(os.listdir(dir_path2))

In [8]:
name2 = dir_path2 + '/zscore_10min_code.xlsx'
df2 = pd.read_excel(name2)

In [9]:
type2 = df2.iloc[:,0]

assign_type2 = []
for i in type2:
    if 'H' in i:
        assign_type2.append(0)
    elif 'R' in i:
        assign_type2.append(1)

assign_type2 = np.array(assign_type2)
assign_type2 = assign_type2.reshape(-1,1)

In [10]:
ns2_1 = df2.iloc[:,1]
ns2_2 = df2.iloc[:,2]
ns2_3 = df2.iloc[:,3]
ns2_4 = df2.iloc[:,4]
ns2_5 = df2.iloc[:,5]

ns2_all = df2.iloc[:,1:6]

ns2_1 = ns2_1.to_numpy().reshape(-1,1)
ns2_2 = ns2_2.to_numpy().reshape(-1,1)
ns2_3 = ns2_3.to_numpy().reshape(-1,1)
ns2_4 = ns2_4.to_numpy().reshape(-1,1)
ns2_5 = ns2_5.to_numpy().reshape(-1,1)

ns2_all = ns2_all.to_numpy()

In [11]:
X_train1_1 = ns1_1
X_train1_2 = ns1_2
X_train1_3 = ns1_3
X_train1_4 = ns1_4
X_train1_5 = ns1_5

X_train1_all = ns1_all
y_train = assign_type

X_test2_1 = ns2_1
X_test2_2 = ns2_2
X_test2_3 = ns2_3
X_test2_4 = ns2_4
X_test2_5 = ns2_5

X_test2_all = ns2_all
y_test = assign_type2.ravel()

In [12]:
clf = RandomForestClassifier(n_estimators=1000, 
                             criterion='gini',
                             random_state=42, 
                             max_depth=10,
                             min_samples_leaf=2,
                            max_features='sqrt')

In [13]:
clf.fit(X_train1_1, y_train)
y_pred1 = clf.predict_proba(X_test2_1)[:, 1]

clf.fit(X_train1_2, y_train)
y_pred2 = clf.predict_proba(X_test2_2)[:, 1]

clf.fit(X_train1_3, y_train)
y_pred3 = clf.predict_proba(X_test2_3)[:, 1]

clf.fit(X_train1_4, y_train)
y_pred4 = clf.predict_proba(X_test2_4)[:, 1]

clf.fit(X_train1_5, y_train)
y_pred5 = clf.predict_proba(X_test2_5)[:, 1]

clf.fit(X_train1_all, y_train)
y_pred_all = clf.predict_proba(X_test2_all)[:, 1]

In [ ]:
from sklearn.metrics import roc_curve, auc
fpr1, tpr1, thresholds = roc_curve(y_test, y_pred1)
fpr2, tpr2, thresholds = roc_curve(y_test, y_pred2)
fpr3, tpr3, thresholds = roc_curve(y_test, y_pred3)
fpr4, tpr4, thresholds = roc_curve(y_test, y_pred4)
fpr5, tpr5, thresholds = roc_curve(y_test, y_pred5)
fpr_all, tpr_all, thresholds = roc_curve(y_test, y_pred_all)

roc_auc1 = auc(fpr1, tpr1)
roc_auc2 = auc(fpr2, tpr2)
roc_auc3 = auc(fpr3, tpr3)
roc_auc4 = auc(fpr4, tpr4)
roc_auc5 = auc(fpr5, tpr5)
roc_auc_all = auc(fpr_all, tpr_all)

#vABN1: BV01-HFA1, vABN2: PP02-HFA3, vABN3: Q6-d5eth, vABN4: PP12-d7isop, vABN5 OCW32-d5but
plt.figure(figsize=(3, 3),dpi = 160)
plt.rc('font', family='Arial')
plt.plot(fpr1, tpr1, color='blue', lw=1.5, label='vABN1 (AUC: %0.2f)' % roc_auc1)
plt.plot(fpr3, tpr3, color='purple', lw=1.5, label='vABN2 (AUC: %0.2f)' % roc_auc3)
plt.plot(fpr5, tpr5, color='red', lw=1.5, label='vABN3 (AUC: %0.2f)' % roc_auc5)
plt.plot(fpr4, tpr4, color='gold', lw=1.5, label='vABN4 (AUC: %0.2f)' % roc_auc4)
plt.plot(fpr2, tpr2, color='green', lw=1.5, label='vABN5 (AUC: %0.2f)' % roc_auc2)

plt.plot(fpr_all, tpr_all, color='Black', lw=1.5, label='Multiplex (AUC: %0.2f)' % roc_auc_all)


plt.plot([0, 1], [0, 1], 'k--', alpha=0.5, lw=1, label='Random classifier')

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)
plt.xlabel('1 - Specificity', fontsize=13)
plt.ylabel('Sensitivity', fontsize=13)
#plt.title('ROC Curve')
plt.legend(bbox_to_anchor=(1.05, 0.9), loc='upper left', edgecolor="None")

plt.show()

In [ ]:
param_dist = {'max_depth': stats.randint(1,20),
             'min_samples_leaf' : stats.randint(1, 5), 
             'random_state': [0, 42],
              'criterion' : ['gini', 'entropy'],
             'max_features' : ['sqrt', 'log2', 0.2, 0.4, 0.6]}

# Create a random forest classifier
rf = RandomForestClassifier()

# Use random search to find the best hyperparameters
rand_search = RandomizedSearchCV(rf, 
                                 param_distributions = param_dist)

In [ ]:
#Randomized search on the training set of data
# Fit the random search object to the data
rand_search.fit(X_train1_all, y_train)
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

In [ ]:
#Randomized search on the testing set of data
# Fit the random search object to the data
rand_search.fit(X_test2_all, y_test)
# Create a variable for the best model
best_rf = rand_search.best_estimator_

# Print the best hyperparameters
print('Best hyperparameters:',  rand_search.best_params_)

In [ ]:
#single plex plot
fpr_all, tpr_all, thresholds = roc_curve(y_test, y_pred_all)
roc_auc_all = auc(fpr_all, tpr_all)
plt.figure(figsize=(3, 3),dpi = 160)
plt.rc('font', family='Arial')
plt.plot(fpr_all, tpr_all, color='black', lw=1.5, label='PR8 (AUC: %0.2f)' % roc_auc_all)
plt.plot([0, 1], [0, 1], 'k--', alpha=0.3, lw=1.2, label='Random classifier')

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel('1 - Specificity', fontsize=13, labelpad=5)
plt.ylabel('Sensitivity', fontsize=13, labelpad=5)
#plt.title('ROC Curve')
plt.legend(bbox_to_anchor=(0.20, 0.25), loc='best', facecolor="None", edgecolor="None",
          fontsize=11)

plt.show()

In [ ]:
#Permute the random state to obtain the average AUC
state = np.random.RandomState(0).randint(0,200,5)
print(state)

AUC_result = []

for i in state: 
    clf = RandomForestClassifier(n_estimators=1000, 
                                 criterion='gini',
                                 random_state=i, 
                                 max_depth=10,
                                 min_samples_leaf=2,
                                max_features='sqrt')

    clf.fit(X_train1_all, y_train)
    y_pred_all = clf.predict_proba(X_test2_all)[:, 1]
    fpr_all, tpr_all, thresholds = roc_curve(y_test, y_pred_all)
    roc_auc_all = auc(fpr_all, tpr_all)
    AUC_result.append(roc_auc_all)

print(AUC_result)
total = sum(AUC_result)
avg = total/5
print(avg)

In [ ]:
y_pred = clf.predict(X_test2_all)
print("ACCURACY OF THE MODEL: ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
#https://matplotlib.org/stable/users/explain/colors/colormaps.html
    
def plot_cm(y_test, y_pred): 
    fig, ax = plt.subplots(figsize=(3, 3),dpi = 160)
    plt.rc('font', family='Arial')
    disp = ConfusionMatrixDisplay.from_predictions(y_test, y_pred, display_labels=['Healthy', 'PR8'],cmap=plt.cm.Greys, ax=ax)
    plt.rcParams.update({'font.size': 11})
    
    label_font = {'size':'12'}  # Adjust to fit
    ax.set_xlabel('Predicted labels', fontdict=label_font, labelpad=5.0);
    ax.set_ylabel('True labels', fontdict=label_font, labelpad=5.0);

In [ ]:
plot_cm(y_test, y_pred)

In [ ]:
#SVC model
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [ ]:
clf = SVC(random_state=0).fit(X_train1_1, y_train)
y_pred1 = clf.decision_function(X_test2_1)
clf = SVC(random_state=0).fit(X_train1_2, y_train)
y_pred2 = clf.decision_function(X_test2_2)
clf = SVC(random_state=0).fit(X_train1_3, y_train)
y_pred3 = clf.decision_function(X_test2_3)
clf = SVC(random_state=0).fit(X_train1_4, y_train)
y_pred4 = clf.decision_function(X_test2_4)
clf = SVC(random_state=0).fit(X_train1_5, y_train)
y_pred5 = clf.decision_function(X_test2_5)
clf = SVC(random_state=0).fit(X_train1_all, y_train)
y_pred_all = clf.decision_function(X_test2_all)

In [ ]:
fpr1, tpr1, thresholds = roc_curve(y_test, y_pred1)
fpr2, tpr2, thresholds = roc_curve(y_test, y_pred2)
fpr3, tpr3, thresholds = roc_curve(y_test, y_pred3)
fpr4, tpr4, thresholds = roc_curve(y_test, y_pred4)
fpr5, tpr5, thresholds = roc_curve(y_test, y_pred5)
fpr_all, tpr_all, thresholds = roc_curve(y_test, y_pred_all)

roc_auc1 = auc(fpr1, tpr1)
roc_auc2 = auc(fpr2, tpr2)
roc_auc3 = auc(fpr3, tpr3)
roc_auc4 = auc(fpr4, tpr4)
roc_auc5 = auc(fpr5, tpr5)
roc_auc_all = auc(fpr_all, tpr_all)


plt.figure(figsize=(3, 3),dpi = 160)
plt.plot(fpr1, tpr1, color='blue', lw=2, label='BV01-HFA1 (area = %0.2f)' % roc_auc1)
plt.plot(fpr2, tpr2, color='green', lw=2, label='OCW32-d3but (area = %0.2f)' % roc_auc2)
plt.plot(fpr3, tpr3, color='purple', lw=2, label='PP02-HFA3 (area = %0.2f)' % roc_auc3)
plt.plot(fpr4, tpr4, color='brown', lw=2, label='PP12-d7isop (area = %0.2f)' % roc_auc4)
plt.plot(fpr5, tpr5, color='hotpink', lw=2, label='Q6-d5eth (area = %0.2f)' % roc_auc5)
plt.plot(fpr_all, tpr_all, color='darkorange', lw=2, label='Multiplex (area = %0.2f)' % roc_auc_all)


plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')

plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlabel('False Positive Rate', fontsize=10)
plt.ylabel('True Positive Rate', fontsize=10)
plt.title('ROC Curve')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
LR = LogisticRegression(random_state=0).fit(X_train1_1, y_train)
LR.fit(X_train1_1, y_train)
y_pred1 = LR.decision_function(X_test2_1)

LR = LogisticRegression(random_state=0).fit(X_train1_2, y_train)
LR.fit(X_train1_2, y_train)
y_pred2 = LR.decision_function(X_test2_2)

LR = LogisticRegression(random_state=0).fit(X_train1_3, y_train)
LR.fit(X_train1_3, y_train)
y_pred3 = LR.decision_function(X_test2_3)

LR = LogisticRegression(random_state=0).fit(X_train1_4, y_train)
LR.fit(X_train1_4, y_train)
y_pred4 = LR.decision_function(X_test2_4)

LR = LogisticRegression(random_state=0).fit(X_train1_5, y_train)
LR.fit(X_train1_5, y_train)
y_pred5 = LR.decision_function(X_test2_5)

LR = LogisticRegression(random_state=0).fit(X_train1_all, y_train)
LR.fit(X_train1_all, y_train)
y_pred_all = LR.decision_function(X_test2_all)

In [ ]:
fpr1, tpr1, thresholds = roc_curve(y_test, y_pred1)
fpr2, tpr2, thresholds = roc_curve(y_test, y_pred2)
fpr3, tpr3, thresholds = roc_curve(y_test, y_pred3)
fpr4, tpr4, thresholds = roc_curve(y_test, y_pred4)
fpr5, tpr5, thresholds = roc_curve(y_test, y_pred5)
fpr_all, tpr_all, thresholds = roc_curve(y_test, y_pred_all)

roc_auc1 = auc(fpr1, tpr1)
roc_auc2 = auc(fpr2, tpr2)
roc_auc3 = auc(fpr3, tpr3)
roc_auc4 = auc(fpr4, tpr4)
roc_auc5 = auc(fpr5, tpr5)
roc_auc_all = auc(fpr_all, tpr_all)


plt.figure(figsize=(3, 3),dpi = 160)
plt.plot(fpr1, tpr1, color='blue', lw=2, label='BV01-HFA1 (area = %0.2f)' % roc_auc1)
plt.plot(fpr2, tpr2, color='green', lw=2, label='OCW32-d3but (area = %0.2f)' % roc_auc2)
plt.plot(fpr3, tpr3, color='purple', lw=2, label='PP02-HFA3 (area = %0.2f)' % roc_auc3)
plt.plot(fpr4, tpr4, color='brown', lw=2, label='PP12-d7isop (area = %0.2f)' % roc_auc4)
plt.plot(fpr5, tpr5, color='hotpink', lw=2, label='Q6-d5eth (area = %0.2f)' % roc_auc5)
plt.plot(fpr_all, tpr_all, color='darkorange', lw=2, label='Multiplex (area = %0.2f)' % roc_auc_all)


plt.plot([0, 1], [0, 1], 'k--', label='Random classifier')

plt.xticks(fontsize=8)
plt.yticks(fontsize=8)
plt.xlabel('False Positive Rate', fontsize=10)
plt.ylabel('True Positive Rate', fontsize=10)
plt.title('ROC Curve')
plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left')

plt.show()